In [4]:
#IMPORTS

import librosa
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import matplotlib
import os

print("Librosa version:", librosa.__version__)
print("Numpy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Python version:", np.__version__)

Librosa version: 0.11.0
Numpy version: 1.26.4
Matplotlib version: 3.10.3
Python version: 1.26.4


In [5]:
def get_spectrogram(path, sr=22050, n_fft=2048, hop_length=512):
    y, _ = librosa.load(path, sr=sr)
    S = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)
    return S_db  # 2D matice (frekvence × čas

In [6]:
def extract_mfcc(path, n_mfcc=13, duration=4.0, sr=22050):
    y, _ = librosa.load(path, sr=sr, duration=duration)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfcc.T, axis=0)  # průměr přes časové okno

In [7]:
import soundata

# Inicializace datasetu
dataset = soundata.initialize("urbansound8k")

# Stáhne dataset, pokud ještě není
print("📦 Kontroluji dataset...")
dataset.download()  

# Načte metadata
dataset.validate()
clip_ids = dataset.clip_ids

print(f"🔍 V databázi je {len(clip_ids)} klipů.")



# Kategorie z UrbanSound8K
target_labels = ['gun_shot']
negative_labels = ['children_playing', 'air_conditioner', 'dog_bark']

X = []
y = []

# 🔊 1. MFCC z UrbanSound8K
for clip_id in dataset.clip_ids:
    clip = dataset.clip(clip_id)
    if clip.tags:
        label = clip.tags.labels[0]
        if label in target_labels + negative_labels:
            try:
                spec = get_spectrogram(clip.audio_path)
                X.append(spec)
                y.append(1 if label == 'gun_shot' else 0)
            except Exception as e:
                print(f"❌ Chyba u UrbanSound {clip.audio_path}: {e}")

# 🔫 2. MFCC z vlastních Freesound WAV souborů
FREESOUND_WAV_DIR = "freesound_gunshots_wav"
for filename in os.listdir(FREESOUND_WAV_DIR):
    if filename.endswith(".wav"):
        try:
            path = os.path.join(FREESOUND_WAV_DIR, filename)
            spec = get_spectrogram(path)
            X.append(spec)
            y.append(1)
        except Exception as e:
            print(f"❌ Chyba u Freesound {filename}: {e}")

# Padding/trimming spectrograms to same shape
max_shape = max([x.shape for x in X], key=lambda s: s[1])
X_pad = []
for spec in X:
    pad_width = max_shape[1] - spec.shape[1]
    if pad_width > 0:
        spec_padded = np.pad(spec, ((0, 0), (0, pad_width)), mode='constant')
    else:
        spec_padded = spec[:, :max_shape[1]]
    X_pad.append(spec_padded)

X = np.array(X_pad)[..., np.newaxis]  # shape: (samples, freq, time, 1)
y = np.array(y)
print("✅ Dataset shape:", X.shape, y.shape)

INFO: Downloading ['all', 'index']. Index is being stored in /home/cigi/soundenv/lib/python3.10/site-packages/soundata/datasets/indexes, and the rest of files in /home/cigi/sound_datasets/urbansound8k
INFO: [all] downloading UrbanSound8K.tar.gz
INFO: /home/cigi/sound_datasets/urbansound8k/UrbanSound8K.tar.gz already exists and will not be downloaded. Rerun with force_overwrite=True to delete this file and force the download.


📦 Kontroluji dataset...


INFO: /home/cigi/sound_datasets/urbansound8k/UrbanSound8K_README.txt already exists. Run with force_overwrite=True to download from scratch
INFO: /home/cigi/sound_datasets/urbansound8k/audio already exists. Run with force_overwrite=True to download from scratch
INFO: /home/cigi/sound_datasets/urbansound8k/FREESOUNDCREDITS.txt already exists. Run with force_overwrite=True to download from scratch
INFO: /home/cigi/sound_datasets/urbansound8k/metadata already exists. Run with force_overwrite=True to download from scratch
INFO: [index] downloading urbansound8k_index_1.0.json
INFO: /home/cigi/soundenv/lib/python3.10/site-packages/soundata/datasets/indexes/urbansound8k_index_1.0.json already exists and will not be downloaded. Rerun with force_overwrite=True to delete this file and force the download.
100%|██████████| 8732/8732 [00:16<00:00, 525.42it/s]
INFO: Success: the dataset is complete and all files are valid.
INFO: --------------------


🔍 V databázi je 8732 klipů.
✅ Dataset shape: (5503, 1025, 174, 1) (5503,)


In [8]:
# Split na tréninkovou a testovací sadu
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
import tensorflow as tf
print("GPU dostupné:", tf.config.list_physical_devices('GPU'))


GPU dostupné: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Model Neurounky 

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Resizing, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D

input_shape = X_train.shape[1:]

model = Sequential([
    Resizing(64, 64, input_shape=input_shape),
    Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.save("gunshot_cnn_model.h5")

ImportError: cannot import name 'GlobalAveragePooling2' from 'tensorflow.keras.layers' (/home/cigi/soundenv/lib/python3.10/site-packages/keras/api/_v2/keras/layers/__init__.py)

In [1]:
from tensorflow.keras.models import load_model

# Načti uložený model
model = load_model("gunshot_cnn_model.h5")

# Ověř si, že je načtený
model.summary()

2025-07-06 15:08:59.436706: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-06 15:08:59.662940: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-06 15:08:59.662987: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-06 15:08:59.688181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-06 15:08:59.753863: I tensorflow/core/platform/cpu_feature_guar

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 128, 128, 1)       0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 32)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 28800)             0

In [ ]:
# Trénink modelu

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=4)

NameError: name 'X_train' is not defined

In [3]:
# Vyhodnocení modelu

loss, acc = model.evaluate(X_test, y_test)
print(f"✅ Test accuracy: {acc:.4f}")

NameError: name 'X_test' is not defined